In [8]:
import pandas as pd 
import numpy as np
import torch
import torch.nn as nn 
import nibabel as nib

from repos.utils import multiprocess

In [9]:
from repos.training.data import create_df
import parameter as para 
from pathlib import Path

In [10]:
root = '/Users/HPhuc/Practice/8. Kits2019/data'

In [11]:
data = create_df.df_image_mask_path(root)
data

,case_id,img_path,seg_path,mask
0,case_00129,/Users/HPhuc/Practice/8. Kits2019/data/case_00...,/Users/HPhuc/Practice/8. Kits2019/data/case_00...,1
1,case_00151,/Users/HPhuc/Practice/8. Kits2019/data/case_00...,/Users/HPhuc/Practice/8. Kits2019/data/case_00...,1
2,case_00190,/Users/HPhuc/Practice/8. Kits2019/data/case_00...,/Users/HPhuc/Practice/8. Kits2019/data/case_00...,1
3,case_00207,/Users/HPhuc/Practice/8. Kits2019/data/case_00...,/Users/HPhuc/Practice/8. Kits2019/data/case_00...,1


In [13]:
import random
idc = np.random.randint(len(data))
data.iloc[0]['seg_path']

'/Users/HPhuc/Practice/8. Kits2019/data/case_00129/segmentation.nii.gz'

In [15]:
data.to_csv(Path(para.csv_path, 'data_df.csv'))


In [16]:
import nibabel as nib 
import numpy as np 
path = data.iloc[0, 1]
img = nib.load(path)
img_arr = nib.load(path).get_fdata()
print(np.diag(abs(img.affine[[2, 1, 0, 3]])))
print(img)


[3.         0.78162497 0.78162497 1.        ]
<class 'nibabel.nifti1.Nifti1Image'>
data shape (80, 482, 482)
affine: 
[[ 0.          0.         -0.78162497  0.        ]
 [ 0.         -0.78162497  0.          0.        ]
 [-3.          0.          0.          0.        ]
 [ 0.          0.          0.          1.        ]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  3  80 482 482   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float64
bitpix          : 64
slice_start     : 0
pixdim          : [1.       3.       0.781625 0.781625 1.       1.       1.       1.      ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min 

slice_percentage

In [ ]:
data = data.sort_values(by=['case_id']).reset_index(drop=True)
data = data.drop_duplicates(keep='first')
data

In [18]:
def single_slice_percent(idx, data):
  row = data.iloc[idx] 
  seg_path = row['seg_path']
  seg = nib.load(seg_path)
  seg_array = seg.get_fdata()

  kidney_slice = 0
  tumor_slice  = 0
  organ_slice  = 0

  for slice_ in seg_array:
    if 1 in slice_ or 2 in slice_: organ_slice += 1
    if 1 in slice_: kidney_slice += 1 
    if 2 in slice_: tumor_slice  += 1 

  return seg_array.shape[0], organ_slice, kidney_slice, tumor_slice

In [20]:
result = multiprocess(single_slice_percent, range(len(data)), data=data)

total_slice        = sum([result[i][0] for i in range(len(result))])
total_organ_slice  = sum([result[i][1] for i in range(len(result))])
total_organ_kidney = sum([result[i][2] for i in range(len(result))])
total_organ_tumor  = sum([result[i][3] for i in range(len(result))])

print('The overall proportion of slices containing the organ in the training set:', total_organ_slice / total_slice)
print('The overall proportion of slices containing the kidney in the training set:', total_organ_kidney / total_slice)
print('The overall proportion of slices containing the tumor in the training set:', total_organ_tumor / total_slice)


...multiprocessing...


100%|██████████| 4/4 [00:03<00:00,  1.03it/s]

The overall proportion of slices containing the organ in the training set: 0.2953846153846154
The overall proportion of slices containing the kidney in the training set: 0.28307692307692306
The overall proportion of slices containing the tumor in the training set: 0.10461538461538461


voxel_percentage


In [21]:
def single_voxel_percent(idx, data):
    row = data.iloc[idx]
    
    seg_path = row['seg_path']
    seg = nib.load(seg_path)
    seg_array = seg.get_fdata()

    # seg_path = row['new_seg_path']
    # seg_array = np.load(seg_path)

    kidney_slice = 0
    tumor_slice  = 0
    organ_slice  = 0

    for slice_ in seg_array:
      if 1 in slice_ or 2 in slice_: 
        organ_slice += 1

    organ_point  = (seg_array  > 0).astype(int).sum()
    kidney_point = (seg_array == 1).astype(int).sum()
    tumor_point  = (seg_array == 2).astype(int).sum()

    organ_slice  = organ_slice * 256 * 256

    return organ_slice, organ_point, kidney_point, tumor_point

In [22]:
result = multiprocess(single_voxel_percent, range(len(data)), data=data)

total_point        = sum([result[i][0] for i in range(len(result))])
total_organ_point  = sum([result[i][1] for i in range(len(result))])
total_kidney_point = sum([result[i][2] for i in range(len(result))])
total_tumor_point  = sum([result[i][3] for i in range(len(result))])

print('The overall proportion of voxels containing the organ in the training set:', total_organ_point / total_point)
print('The overall proportion of voxels containing the kidney in the training set:', total_kidney_point / total_point)
print('The overall proportion of voxels containing the tumor in the training set:', total_tumor_point / total_point)

...multiprocessing...


100%|██████████| 4/4 [00:03<00:00,  1.19it/s]

The overall proportion of voxels containing the organ in the training set: 0.09286785125732422
The overall proportion of voxels containing the kidney in the training set: 0.08012922604878743
The overall proportion of voxels containing the tumor in the training set: 0.012738625208536783


Get_threshold

In [23]:
def get_threshold(idx, data):
    row = data.iloc[idx]
    ct_path = row['img_path']
    ct      = nib.load(ct_path)
    ct_array = ct.get_fdata()

    seg_path = row['seg_path']
    seg      = nib.load(seg_path)
    seg_array = seg.get_fdata()

    organ_roi = ct_array[seg_array > 0]
    inliers = ((organ_roi < para.upper_bound) * (organ_roi > para.lower_bound)).astype(int).sum()

    organ_roi_shape = organ_roi.shape[0]

    return organ_roi_shape, inliers

In [24]:
result = multiprocess(get_threshold, range(len(data)), data=data)

num_point        = sum([result[i][0] for i in range(len(result))])
num_inlier       = sum([result[i][1] for i in range(len(result))])
print("Ngưỡng threshold bound = [{}, {}]  đối với kidney trong tập huấn luyện".format(para.lower_bound, para.upper_bound))
print(num_inlier / num_point)

...multiprocessing...


100%|██████████| 4/4 [00:05<00:00,  1.26s/it]

Ngưỡng threshold bound = [-79, 304]  đối với kidney trong tập huấn luyện
0.9920140208189993


Get_spacing